In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Mon Jun 29 10:41:52 PDT 2020


# NOTE: Conclusion statements in this notebook have not been updated

In [2]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/nigeria/2020_06_26_20_28_27/count_data/']

locations = ['Nigeria']

# 1. Iron effect on birth weight

In [3]:
births = pd.read_hdf(output_dirs[0] + 'births.hdf')
births.head()

# note: no stratification of birth counts by iron coverage group

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,21,baseline,3325.0
1,2020,female,covered,live_births,21,baseline,5.0
2,2020,female,covered,live_births,21,baseline,249.0
3,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,3325.0
4,2020,female,covered,live_births,21,folic_acid_fortification_scale_up,5.0


In [4]:
bw = pd.read_hdf(output_dirs[0] + 'birth_weight.hdf')
bw.head()

,year,sex,measure,input_draw,scenario,value,iron_fortification_group
0,2020,female,birth_weight_mean,21,baseline,3197.119032,uncovered
1,2020,female,birth_weight_mean,21,baseline,3188.722066,covered
2,2020,female,birth_weight_mean,21,folic_acid_fortification_scale_up,3197.119032,uncovered
3,2020,female,birth_weight_mean,21,folic_acid_fortification_scale_up,3188.722066,covered
4,2020,female,birth_weight_mean,21,iron_folic_acid_fortification_scale_up,3197.119032,uncovered


In [5]:
bw_by_coverage_and_scenario = bw.loc[bw.measure == 'birth_weight_mean']
bw_by_coverage_and_scenario = bw_by_coverage_and_scenario.groupby(['scenario','iron_fortification_group']).mean()
bw_by_coverage_and_scenario

# we would expect mean birth weight to be higher in covered group than uncovered group (for all scenarios)
# we are seeing the opposite
# but Kjell verified that the MEDIAN birth weight in covered group > median birth weight in uncovered group
    # this suggests that the distribution is differentially skewed high
    
# we would also expect to see simular mean birth weights by coverage group between scenarios
    # we are not seeing this.

input_draw  \
scenario                               iron_fortification_group               
baseline                               covered                       437.52   
                                       uncovered                     437.52   
folic_acid_fortification_scale_up      covered                       437.52   
                                       uncovered                     437.52   
iron_folic_acid_fortification_scale_up covered                       437.52   
                                       uncovered                     437.52   
vitamin_a_fortification_scale_up       covered                       437.52   
                                       uncovered                     437.52   

                                                                       value  
scenario                               iron_fortification_group               
baseline                               covered                   3183.760907  
                                       uncovered                 3184.004750  
folic_acid_fortification_scale_up      covered                   3183.760907  
                                       uncovered                 3184.004750  
iron_folic_acid_fortification_scale_up covered                   3183.784201  
                                       uncovered                 3183.963375  
vitamin_a_fortification_scale_up       covered                   3183.760907  
                                       uncovered                 3184.004750

## Birth weight conclusions

- Cannot calculate overall mean birth weight by scenario because births are not stratified by iron coverage group
- Mean birthweight is lower in the covered group than the uncovered group... this is the opposite of what we would expect
    - Kjell verified that the MEDIAN birth weight is higher in the covered group than uncovered group, indicating that this is likely working as expected
- Mean birth weight for each covered group should be approximately equal between scenarios, although we are not seeing this behavior

**So, this is tentatively thought to be working as intended**

# 2. Vitamin A Fortification and Vitamin A Deficiency Prevalence

In [6]:
state_pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
state_pt.head()

,year,age_group,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,baseline,0.0
1,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,diarrheal_diseases,covered,covered,person_time,29,baseline,0.0


In [7]:
def calculate_stratified_vad_prevalence(strata_cols):
    state_pt = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
    vad = state_pt.loc[state_pt.cause == 'vitamin_a_deficiency']
    vad = vad.groupby((['scenario','input_draw'] + strata_cols)).sum()
    pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    vad_prev = vad / pt
    vad_prev = vad_prev.reset_index()
    vad_prev = vad_prev.loc[vad_prev.scenario != 'iron_fortification_scale_up']
    vad_prev = (vad_prev.groupby((['scenario'] + strata_cols)).mean()
                .rename(columns={'value':'vad_prevalence'})
                .drop(columns='input_draw'))
    return vad_prev

In [8]:
overall = calculate_stratified_vad_prevalence([])
overall

# looks good! (Prevalence of VAD is lower in vitamin_a_fortification_scale_up scenario than baseline scenario)

,vad_prevalence
scenario,
baseline,0.001194
folic_acid_fortification_scale_up,0.001194
iron_folic_acid_fortification_scale_up,0.001194
vitamin_a_fortification_scale_up,0.001098


In [9]:
by_year = calculate_stratified_vad_prevalence(['year'])
by_year

# looks good! 
# VAD prevalence is the same in each scenario in 2020, and then lower in vit_a_scenaario in 2021-2025

vad_prevalence
scenario                               year                
baseline                               2020        0.000511
                                       2021        0.001000
                                       2022        0.002700
                                       2023        0.018761
folic_acid_fortification_scale_up      2020        0.000511
                                       2021        0.001000
                                       2022        0.002700
                                       2023        0.018761
iron_folic_acid_fortification_scale_up 2020        0.000511
                                       2021        0.001000
                                       2022        0.002700
                                       2023        0.018762
vitamin_a_fortification_scale_up       2020        0.000511
                                       2021        0.000948
                                       2022        0.002384
                                       2023        0.015960

In [10]:
by_age = calculate_stratified_vad_prevalence(['age_group'])
by_age

# no change in neonatal age groups between scenarios, as expected
# lower VAD prevalence in vitamin_a_scenario in post_neonatal and 1_to_4 ages, as expected
# looks good!

vad_prevalence
scenario                               age_group                     
baseline                               1_to_4                0.000980
                                       early_neonatal        0.007270
                                       late_neonatal         0.009929
                                       post_neonatal         0.012717
folic_acid_fortification_scale_up      1_to_4                0.000980
                                       early_neonatal        0.007271
                                       late_neonatal         0.009930
                                       post_neonatal         0.012718
iron_folic_acid_fortification_scale_up 1_to_4                0.000980
                                       early_neonatal        0.007272
                                       late_neonatal         0.009931
                                       post_neonatal         0.012718
vitamin_a_fortification_scale_up       1_to_4                0.000893
                                       early_neonatal        0.007288
                                       late_neonatal         0.009964
                                       post_neonatal         0.012153

In [11]:
by_coverage_group = calculate_stratified_vad_prevalence(['vitamin_a_fortification_group'])
by_coverage_group

# unexpected results here...

# we should expect...

    # all VAD_prev < 1 in each coverage group
    # covered VAD_prev ~= uncovered VAD_prev
    # effectively_covered VAD_prev * 2.22 ~= uncovered VAD_prev in both scenarios
    # all VAD_prev by coverage group in baseline scenario should ~= VAD_prev by coverage group in vitamin_a scenario

vad_prevalence
scenario                               vitamin_a_fortification_group                
baseline                               covered                              0.036559
                                       effectively_covered                  0.000555
                                       uncovered                            0.001235
folic_acid_fortification_scale_up      covered                              0.036562
                                       effectively_covered                  0.000555
                                       uncovered                            0.001235
iron_folic_acid_fortification_scale_up covered                              0.036563
                                       effectively_covered                  0.000555
                                       uncovered                            0.001235
vitamin_a_fortification_scale_up       covered                              0.014500
                                       effectively_covered                  0.000472
                                       uncovered                            0.001254

## Conclusions

- For the covered group, VAD state person time > overall person time, which appears to be an error (prevalence > 1)
- For the baseline scenario, VAD prevalence in the effectively covered group is ~ 0.45 times that in the uncovered group, as expected. However, in the vitamin A scenario this is not the case and the difference is greater between these groups -- we would expect to see the same rates here and just more person time in the covered groups in the vitamin A scenario.
- Otherwise, behavior by year, age group, and overall looks as expected
- VAD prevalence does not decrease until 2021, indicating that the coverage algorithm was implemented correctly

# 3. Iron effect on hemoglobin

In [12]:
hb = pd.read_hdf(output_dirs[0] + 'hemoglobin_level.hdf')
hb.head()

,sex,measure,input_draw,scenario,value,age,status,responsive
0,female,hemoglobin_mean,21,baseline,99.861305,0.5,covered,responsive
1,female,hemoglobin_mean,21,baseline,89.421314,0.5,covered,non-responsive
2,female,hemoglobin_mean,21,baseline,100.046889,0.5,uncovered,responsive
3,female,hemoglobin_mean,21,baseline,92.766277,0.5,uncovered,non-responsive
4,female,hemoglobin_mean,21,baseline,112.162315,1,covered,responsive


In [13]:
np.unique(hb['measure'])

array(['hemoglobin_mean', 'hemoglobin_variance'], dtype=object)

In [14]:
pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
pt.head()

# NOTE: cannot calculate weighted averages of Hb because persontime data not stratified by iron coverage or responsiveness

,year,age_group,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,covered,covered,person_time,21,baseline,0.0
1,2020,1_to_4,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,covered,covered,person_time,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,covered,covered,person_time,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,covered,covered,person_time,29,baseline,0.0


In [15]:
def get_stratified_hb_mean(strata_cols):
    hb = pd.read_hdf(output_dirs[0] + 'hemoglobin_level.hdf')
    hb = hb.loc[hb.scenario != 'vitamin_a_fortification_scale_up']
    hb = hb.loc[hb.measure == 'hemoglobin_mean']
    hb = hb.groupby(['scenario','measure'] + strata_cols).mean()
    return hb

In [16]:
overall = get_stratified_hb_mean([])
overall


# looks good! (mean hemoglobin is slightly higher in iron_fortification scenario)

,,input_draw,value
scenario,measure,,
baseline,hemoglobin_mean,437.52,97.737758
folic_acid_fortification_scale_up,hemoglobin_mean,437.52,97.737754
iron_folic_acid_fortification_scale_up,hemoglobin_mean,437.52,98.191855


In [17]:
by_coverage = get_stratified_hb_mean(['status'])
by_coverage

# hemoglobin is lower in covered group, which is the opposite of what we would expect

# should expect...
    
    # covered mean hemoglobin to be higher than uncovered mean hemoglobin in both scenarios
        # note, if this is not true, it should be true for MEDIAN hemoglobin by coverage group
    # mean hemoglobin by coverage group should be approximately equal between scenarios
        # note, it is ok if covered hemoglobin in the iron scenario is slightly less than the covered 
        # group in the baseline scenario

input_draw  \
scenario                               measure         status                  
baseline                               hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   
folic_acid_fortification_scale_up      hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean covered        437.52   
                                                       uncovered      437.52   

                                                                      value  
scenario                               measure         status                
baseline                               hemoglobin_mean covered    97.321805  
                                                       uncovered  98.153712  
folic_acid_fortification_scale_up      hemoglobin_mean covered    97.321805  
                                                       uncovered  98.153703  
iron_folic_acid_fortification_scale_up hemoglobin_mean covered    98.247857  
                                                       uncovered  98.135853

In [18]:
by_responsiveness = get_stratified_hb_mean(['status','responsive'])
by_responsiveness

# we are seeing some unexpected results here

    
# expected results:

    # for a given scenario...
        # covered, responsive hemoglobin > uncovered, responsive hemoglobin (by ~ 3 units)
        # covered, non-responsive hemoglobin ~= uncovered, non-responsive hemoglobin
    
    # mean hemoglobin by coverage group and responsiveness should be approximately equal between scenarios
        
    # for a given coverage group, responsive hemoglobin > non-responsive hemoglobin
        # for a given ANEMIA group (i.e. severe anemia)... responsive hemoglobin ~= non-responsive hemoglobin
            # can verify this in interactive sim?
        # note: difference between responsive and non-responsive should be larger in covered group than uncovered group

input_draw  \
scenario                               measure         status    responsive                   
baseline                               hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   
folic_acid_fortification_scale_up      hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean covered   non-responsive      437.52   
                                                                 responsive          437.52   
                                                       uncovered non-responsive      437.52   
                                                                 responsive          437.52   

                                                                                      value  
scenario                               measure         status    responsive                  
baseline                               hemoglobin_mean covered   non-responsive   90.624500  
                                                                 responsive      104.019109  
                                                       uncovered non-responsive   93.632450  
                                                                 responsive      102.674975  
folic_acid_fortification_scale_up      hemoglobin_mean covered   non-responsive   90.624500  
                                                                 responsive      104.019109  
                                                       uncovered non-responsive   93.632452  
                                                                 responsive      102.674954  
iron_folic_acid_fortification_scale_up hemoglobin_mean covered   non-responsive   92.357642  
                                                                 responsive      104.138072  
                                                       uncovered non-responsive   93.598190  
                                                                 responsive      102.673516

In [19]:
by_age = get_stratified_hb_mean(['age'])
by_age

# would expect to see slight difference for 0.5 age group between scenarios
# expect to see larger differences between scenarios for other ages

input_draw  \
scenario                               measure         age               
baseline                               hemoglobin_mean 0.5      437.52   
                                                       1        437.52   
                                                       2        437.52   
                                                       3        437.52   
folic_acid_fortification_scale_up      hemoglobin_mean 0.5      437.52   
                                                       1        437.52   
                                                       2        437.52   
                                                       3        437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean 0.5      437.52   
                                                       1        437.52   
                                                       2        437.52   
                                                       3        437.52   

                                                                value  
scenario                               measure         age             
baseline                               hemoglobin_mean 0.5  94.961783  
                                                       1    98.429911  
                                                       2    98.796553  
                                                       3    98.762786  
folic_acid_fortification_scale_up      hemoglobin_mean 0.5  94.961771  
                                                       1    98.429905  
                                                       2    98.796553  
                                                       3    98.762786  
iron_folic_acid_fortification_scale_up hemoglobin_mean 0.5  95.461097  
                                                       1    98.884701  
                                                       2    99.221204  
                                                       3    99.200418

In [20]:
by_responsiveness = get_stratified_hb_mean(['responsive'])
by_responsiveness

# we would expect...

    # baseline non-responsive hemoglobin ~= iron scenario non-responsive hemoglobin
    # baseline responsive hemoglobin < iron scenario responsive hemoglobin (by < 3 units)
    
# we are seeing unexpected results here in that ...

    # baseline non-responsive hemoglobin < iron scenario non-responsive hemoglobin
    # baseline responsive hemoglobin = iron scenario responsive hemoglobin

input_draw  \
scenario                               measure         responsive                   
baseline                               hemoglobin_mean non-responsive      437.52   
                                                       responsive          437.52   
folic_acid_fortification_scale_up      hemoglobin_mean non-responsive      437.52   
                                                       responsive          437.52   
iron_folic_acid_fortification_scale_up hemoglobin_mean non-responsive      437.52   
                                                       responsive          437.52   

                                                                            value  
scenario                               measure         responsive                  
baseline                               hemoglobin_mean non-responsive   92.128475  
                                                       responsive      103.347042  
folic_acid_fortification_scale_up      hemoglobin_mean non-responsive   92.128476  
                                                       responsive      103.347031  
iron_folic_acid_fortification_scale_up hemoglobin_mean non-responsive   92.977916  
                                                       responsive      103.405794

## Conclusions

- hemoglobin is lower in covered group, which is the opposite of what we would expect
- there is significant variation in mean hemoglobin between responsive and non-responsive groups in the uncovered population, which is unexpected
- seeing a difference in mean hemoglobin between scenarios for the 0.5 age, which is unexpected
- seeing an increase in mean hemoglobin between scenarios in the non-responsive population, which is unexpected

# 4. Iron fortification on anemia prevalence

In [21]:
anemia_pt = pd.read_hdf(output_dirs[0] + 'anemia_state_person_time.hdf')
anemia_pt.head()

,year,age_group,sex,measure,input_draw,scenario,value
0,2020,1_to_4,female,anemia_mild,21,baseline,35566.425736
1,2020,1_to_4,female,anemia_mild,21,folic_acid_fortification_scale_up,35566.425736
2,2020,1_to_4,female,anemia_mild,21,iron_folic_acid_fortification_scale_up,35566.425736
3,2020,1_to_4,female,anemia_mild,21,vitamin_a_fortification_scale_up,35566.425736
4,2020,1_to_4,female,anemia_mild,29,baseline,41695.101985


In [22]:
def get_anemia_prevalence(strata_cols):
    anemia_pt = pd.read_hdf(output_dirs[0] + 'anemia_state_person_time.hdf')
    anemia_pt = anemia_pt.loc[anemia_pt.scenario != 'vitamin_a_fortification_scale_up']
    anemia_pt = anemia_pt.groupby((['scenario','input_draw'] + strata_cols + ['measure'])).sum()
    pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
    pt = pt.loc[pt.scenario != 'vitamin_a_fortification_scale_up']
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    anemia_prev = anemia_pt / pt
    anemia_prev = anemia_prev.groupby((['scenario','measure'] + strata_cols)).mean() 
    return anemia_prev

In [23]:
overall = get_anemia_prevalence([])
overall

# we are seeing a VERY slight decrease in anemia prevalence between scenarios
# this is generally expected
# should see decrease in all anemia prevalences in iron scenario relative to baseline
    # an increase in anemia_none prevalence

value
scenario                               measure                  
baseline                               anemia_mild      0.001320
                                       anemia_moderate  0.002020
                                       anemia_none      0.001712
                                       anemia_severe    0.000188
folic_acid_fortification_scale_up      anemia_mild      0.001320
                                       anemia_moderate  0.002020
                                       anemia_none      0.001712
                                       anemia_severe    0.000188
iron_folic_acid_fortification_scale_up anemia_mild      0.001318
                                       anemia_moderate  0.002001
                                       anemia_none      0.001736
                                       anemia_severe    0.000186

In [24]:
by_year = get_anemia_prevalence(['year'])
by_year = by_year.reset_index().sort_values(by=['measure','year','scenario']).set_index(['measure','year','scenario'])
by_year

# this is displaying expected behavior
# no difference in 2020, slight differences in later years

value
measure         year scenario                                        
anemia_mild     2020 baseline                                0.000563
                     folic_acid_fortification_scale_up       0.000563
                     iron_folic_acid_fortification_scale_up  0.000563
                2021 baseline                                0.001105
                     folic_acid_fortification_scale_up       0.001105
                     iron_folic_acid_fortification_scale_up  0.001102
                2022 baseline                                0.002987
                     folic_acid_fortification_scale_up       0.002987
                     iron_folic_acid_fortification_scale_up  0.002979
                2023 baseline                                0.020803
                     folic_acid_fortification_scale_up       0.020804
                     iron_folic_acid_fortification_scale_up  0.020748
anemia_moderate 2020 baseline                                0.000864
                     folic_acid_fortification_scale_up       0.000864
                     iron_folic_acid_fortification_scale_up  0.000864
                2021 baseline                                0.001690
                     folic_acid_fortification_scale_up       0.001690
                     iron_folic_acid_fortification_scale_up  0.001671
                2022 baseline                                0.004568
                     folic_acid_fortification_scale_up       0.004568
                     iron_folic_acid_fortification_scale_up  0.004509
                2023 baseline                                0.031766
                     folic_acid_fortification_scale_up       0.031767
                     iron_folic_acid_fortification_scale_up  0.031371
anemia_none     2020 baseline                                0.000728
                     folic_acid_fortification_scale_up       0.000728
                     iron_folic_acid_fortification_scale_up  0.000728
                2021 baseline                                0.001432
                     folic_acid_fortification_scale_up       0.001432
                     iron_folic_acid_fortification_scale_up  0.001457
                2022 baseline                                0.003879
                     folic_acid_fortification_scale_up       0.003879
                     iron_folic_acid_fortification_scale_up  0.003955
                2023 baseline                                0.027037
                     folic_acid_fortification_scale_up       0.027038
                     iron_folic_acid_fortification_scale_up  0.027545
anemia_severe   2020 baseline                                0.000081
                     folic_acid_fortification_scale_up       0.000081
                     iron_folic_acid_fortification_scale_up  0.000081
                2021 baseline                                0.000158
                     folic_acid_fortification_scale_up       0.000158
                     iron_folic_acid_fortification_scale_up  0.000155
                2022 baseline                                0.000426
                     folic_acid_fortification_scale_up       0.000426
                     iron_folic_acid_fortification_scale_up  0.000418
                2023 baseline                                0.002957
                     folic_acid_fortification_scale_up       0.002957
                     iron_folic_acid_fortification_scale_up  0.002903

In [25]:
by_age = get_anemia_prevalence(['age_group'])
by_age = by_age.reset_index().sort_values(by=['measure','age_group','scenario']).set_index(['measure','age_group','scenario'])
by_age

# this is showing expected behavior... no change in early/late neonatal age groups, but slight change in others

value
measure         age_group      scenario                                        
anemia_mild     1_to_4         baseline                                0.001078
                               folic_acid_fortification_scale_up       0.001078
                               iron_folic_acid_fortification_scale_up  0.001075
                early_neonatal baseline                                0.003076
                               folic_acid_fortification_scale_up       0.003077
                               iron_folic_acid_fortification_scale_up  0.003078
                late_neonatal  baseline                                0.002522
                               folic_acid_fortification_scale_up       0.002522
                               iron_folic_acid_fortification_scale_up  0.002523
                post_neonatal  baseline                                0.015329
                               folic_acid_fortification_scale_up       0.015330
                               iron_folic_acid_fortification_scale_up  0.015335
anemia_moderate 1_to_4         baseline                                0.001519
                               folic_acid_fortification_scale_up       0.001519
                               iron_folic_acid_fortification_scale_up  0.001500
                early_neonatal baseline                                0.025208
                               folic_acid_fortification_scale_up       0.025213
                               iron_folic_acid_fortification_scale_up  0.025217
                late_neonatal  baseline                                0.035202
                               folic_acid_fortification_scale_up       0.035206
                               iron_folic_acid_fortification_scale_up  0.035208
                post_neonatal  baseline                                0.027497
                               folic_acid_fortification_scale_up       0.027498
                               iron_folic_acid_fortification_scale_up  0.027470
anemia_none     1_to_4         baseline                                0.001489
                               folic_acid_fortification_scale_up       0.001489
                               iron_folic_acid_fortification_scale_up  0.001512
                early_neonatal baseline                                0.000108
                               folic_acid_fortification_scale_up       0.000109
                               iron_folic_acid_fortification_scale_up  0.000109
                late_neonatal  baseline                                0.000045
                               folic_acid_fortification_scale_up       0.000045
                               iron_folic_acid_fortification_scale_up  0.000045
                post_neonatal  baseline                                0.015026
                               folic_acid_fortification_scale_up       0.015027
                               iron_folic_acid_fortification_scale_up  0.015057
anemia_severe   1_to_4         baseline                                0.000131
                               folic_acid_fortification_scale_up       0.000131
                               iron_folic_acid_fortification_scale_up  0.000129
                early_neonatal baseline                                0.003485
                               folic_acid_fortification_scale_up       0.003485
                               iron_folic_acid_fortification_scale_up  0.003486
                late_neonatal  baseline                                0.006305
                               folic_acid_fortification_scale_up       0.006305
                               iron_folic_acid_fortification_scale_up  0.006306
                post_neonatal  baseline                                0.002868
                               folic_acid_fortification_scale_up       0.002868
                               iron_folic_acid_fortification_scale_up  0.002862

## Conclusions

- Nothing unexpected here!

# 5. VAD and affected causes

In [26]:
deaths = pd.read_hdf(output_dirs[0] + 'deaths.hdf')
deaths.head()

,year,age_group,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,baseline,0.0
1,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,diarrheal_diseases,covered,covered,death,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,diarrheal_diseases,covered,covered,death,29,baseline,0.0


In [27]:
np.unique(deaths['cause'])

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'other_causes'], dtype=object)

In [28]:
# vad should affect LRI, measles, DD, but NOT NTDs

In [29]:
def get_csmrs(strata_cols):
    deaths = pd.read_hdf(output_dirs[0] + 'deaths.hdf')
    pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
    deaths = deaths.groupby((['scenario','input_draw'] + strata_cols + ['cause'])).sum()
    pt = pt.groupby((['scenario','input_draw'] + strata_cols)).sum()
    csmrs = deaths / pt
    csmrs = csmrs.reset_index()
    csmrs = csmrs.loc[csmrs.scenario != 'iron_fortification_scale_up']
    csmrs = csmrs.groupby((['cause'] + strata_cols + ['scenario'])).mean()
    return csmrs

In [30]:
overall = get_csmrs([])
overall

# looks good!
# CSMR for other causes may be slightly higher in vitamin A scenario because 
# simulants who did not die of affected causes can now die of another cause

input_draw  \
cause                        scenario                                             
diarrheal_diseases           baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
measles                      baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
neural_tube_defects          baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   
other_causes                 baseline                                    437.52   
                             folic_acid_fortification_scale_up           437.52   
                             iron_folic_acid_fortification_scale_up      437.52   
                             vitamin_a_fortification_scale_up            437.52   

                                                                        value  
cause                        scenario                                          
diarrheal_diseases           baseline                                0.000017  
                             folic_acid_fortification_scale_up       0.000017  
                             iron_folic_acid_fortification_scale_up  0.000017  
                             vitamin_a_fortification_scale_up        0.000016  
lower_respiratory_infections baseline                                0.000019  
                             folic_acid_fortification_scale_up       0.000019  
                             iron_folic_acid_fortification_scale_up  0.000019  
                             vitamin_a_fortification_scale_up        0.000019  
measles                      baseline                                0.000003  
                             folic_acid_fortification_scale_up       0.000003  
                             iron_folic_acid_fortification_scale_up  0.000003  
                             vitamin_a_fortification_scale_up        0.000003  
neural_tube_defects          baseline                                0.000001  
                             folic_acid_fortification_scale_up       0.000001  
                             iron_folic_acid_fortification_scale_up  0.000001  
                             vitamin_a_fortification_scale_up        0.000001  
other_causes                 baseline                                0.000071  
                             folic_acid_fortification_scale_up       0.000071  
                             iron_folic_acid_fortification_scale_up  0.000071  
                             vitamin_a_fortification_scale_up        0.000071

In [31]:
by_age = get_csmrs(['age_group'])
by_age

# this is behaving as expected
# lower CSMRs for postneonatal and 1-4 age group, but not neonatal age groups

input_draw  \
cause                        age_group      scenario                                             
diarrheal_diseases           1_to_4         baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             post_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections 1_to_4         baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             post_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
...                                                                                        ...   
neural_tube_defects          1_to_4         iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             early_neonatal baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortification_scale_up      437.52   
                                            vitamin_a_fortification_scale_up            437.52   
                             late_neonatal  baseline                                    437.52   
                                            folic_acid_fortification_scale_up           437.52   
                                            iron_folic_acid_fortif

In [32]:
by_year = get_csmrs(['year'])
by_year

# this is behaving as we would expect with the CORRECT population coverage algorithm (no change until 2020)

input_draw  \
cause                        year scenario                                             
diarrheal_diseases           2020 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2021 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2022 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2023 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
lower_respiratory_infections 2020 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2021 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2022 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2023 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
...                                                                              ...   
neural_tube_defects          2020 iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2021 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2022 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                                  vitamin_a_fortification_scale_up            437.52   
                             2023 baseline                                    437.52   
                                  folic_acid_fortification_scale_up           437.52   
                                  iron_folic_acid_fortification_scale_up      437.52   
                          

## Conclusions

This is looking as expected (VAD prevalence appears to be affecting CSMRs due to diarrheal diseases, measles, and LRI.

However, it also appears to be affecting "other causes" which is unexpected